### 开始

In [1]:
# 检查Python版本
from sys import version_info
if version_info.major != 3:
    raise Exception('请使用Python3来完成此项目')

### 数据预处理

#### 获取数据
[Dogs vs. Cats Redux: Kernels Edition
](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

In [6]:
# download data and unzip it
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

train_url = "https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip"
test_url = "https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/test.zip"

def download_dataset(url, zip_file_path, folder_path):
    if os.path.exists(zip_file_path):
        print("file is exist, no need download\n")
    else:
        print("download now\n")
        urlretrieve(url, zip_file_path)
        
    if os.path.exists(folder_path):
        print("files found\n")
    else:
        print("unzip now\n")
        zipf = ZipFile(zip_file_path)
        zipf.extractall()

download_dataset(train_url, 'train.zip', 'train/')
download_dataset(test_url, 'test.zip', 'test/')

file is exist, no need download

unzip now



#### 分离数据集，dog和cat图片分别放入train2/dogs, train2/cats

In [4]:
import os
import shutil

def split_train_set(old_dir, new_dir):
    file_list = os.listdir(old_dir)
    file_cats = filter(lambda x:x[:3] == 'cat', file_list)
    file_dogs = filter(lambda x:x[:3] == 'dog', file_list)

    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
    os.mkdir(new_dir)
    
    dogs_path = os.path.join(new_dir, 'dogs')
    cats_path = os.path.join(new_dir, 'cats')
    os.mkdir(dogs_path)
    os.mkdir(cats_path)
    
    for filename in file_cats:
        os.symlink(old_dir+filename, cats_path+filename)
    
    for filename in file_dogs:
        os.symlink(old_dir+filename, dogs_path+filename)

split_train_set('train', 'train2')

FileNotFoundError: [Errno 2] No such file or directory: 'train'